# 05 - Agent Weather

学习如何创建智能体（Agent），让 AI 能够使用工具获取实时天气信息并生成可视化图表。

## 学习目标
- 理解 Agent 的工作原理
- 掌握自定义工具的创建
- 学习 LangChain 1.x 的工具调用机制
- 实现实时数据获取和可视化

In [ ]:
# 导入必要的库
import os
import json
import requests
from datetime import datetime, timedelta
from dotenv import load_dotenv
from pydantic import SecretStr

# 可视化库
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
matplotlib.rcParams['axes.unicode_minus'] = False

# 加载环境变量
load_dotenv(override=True)

# 检查 API 密钥
if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("请设置 OPENAI_API_KEY 环境变量")

# 导入 LangChain 组件
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage

print("✓ 环境和组件导入完成")

## 1. 创建天气获取工具

In [ ]:
# 模拟天气 API（实际项目中需要使用真实的天气 API）
def get_weather_data(location: str, days: int = 1) -> dict:
    """获取天气数据（模拟）"""
    # 模拟不同城市的天气数据
    weather_database = {
        "北京": {
            "temp_range": (15, 25),
            "conditions": ["晴", "多云", "小雨"],
            "humidity_range": (40, 70),
            "wind_range": (5, 15)
        },
        "上海": {
            "temp_range": (18, 28),
            "conditions": ["多云", "阴", "小雨"],
            "humidity_range": (60, 80),
            "wind_range": (10, 20)
        },
        "广州": {
            "temp_range": (22, 32),
            "conditions": ["晴", "多云", "雷阵雨"],
            "humidity_range": (70, 90),
            "wind_range": (5, 12)
        },
        "深圳": {
            "temp_range": (23, 31),
            "conditions": ["晴", "多云", "阵雨"],
            "humidity_range": (65, 85),
            "wind_range": (8, 15)
        }
    }

    # 获取城市信息，如果城市不存在则使用默认数据
    city_data = weather_database.get(location, {
        "temp_range": (10, 20),
        "conditions": ["晴", "多云", "阴"],
        "humidity_range": (50, 70),
        "wind_range": (5, 15)
    })

    # 生成模拟天气数据
    weather_data = []
    base_date = datetime.now()

    for i in range(days):
        date = base_date + timedelta(days=i)
        temp_min, temp_max = city_data["temp_range"]
        humidity_min, humidity_max = city_data["humidity_range"]
        wind_min, wind_max = city_data["wind_range"]

        # 添加一些随机性
        import random
        random.seed(hash(location + str(i)))  # 确保同一城市同一天的数据一致

        day_data = {
            "date": date.strftime("%Y-%m-%d"),
            "location": location,
            "temperature": {
                "min": round(temp_min + random.uniform(-2, 2), 1),
                "max": round(temp_max + random.uniform(-2, 2), 1),
                "avg": round((temp_min + temp_max) / 2 + random.uniform(-1, 1), 1)
            },
            "condition": random.choice(city_data["conditions"]),
            "humidity": round(random.uniform(humidity_min, humidity_max), 1),
            "wind_speed": round(random.uniform(wind_min, wind_max), 1),
            "rain": random.choice([True, False]) if "雨" in random.choice(city_data["conditions"]) else False
        }
        weather_data.append(day_data)

    return {
        "location": location,
        "days": days,
        "data": weather_data
    }

print("✓ 天气数据模拟函数创建完成")

In [ ]:
# 使用 @tool 装饰器创建天气获取工具
@tool
def get_weather(location: str, days: int = 1) -> str:
    """获取指定地点未来几天的天气信息。

    Args:
        location (str): 城市名称，如"北京"、"上海"
        days (int): 查询天数，默认1天，最多7天

    Returns:
        str: 天气信息的JSON格式字符串
    """
    # 限制查询天数
    days = min(max(days, 1), 7)

    try:
        weather_data = get_weather_data(location, days)
        return json.dumps(weather_data, ensure_ascii=False, indent=2)
    except Exception as e:
        return json.dumps({"error": f"获取天气数据失败：{str(e)}"}, ensure_ascii=False)

print("✓ 天气获取工具创建完成")
print(f"工具名称：{get_weather.name}")
print(f"工具描述：{get_weather.description}")

## 2. 创建天气可视化工具

In [ ]:
# 创建可视化工具
@tool
def visualize_weather(weather_json: str) -> str:
    """根据天气数据生成可视化图表。

    Args:
        weather_json (str): 天气数据的JSON字符串

    Returns:
        str: 图表文件路径或描述信息
    """
    try:
        # 解析天气数据
        weather_data = json.loads(weather_json)

        if "error" in weather_data:
            return f"无法可视化：{weather_data['error']}"

        # 提取数据
        data = weather_data["data"]
        dates = [d["date"] for d in data]
        temps_min = [d["temperature"]["min"] for d in data]
        temps_max = [d["temperature"]["max"] for d in data]
        temps_avg = [d["temperature"]["avg"] for d in data]
        humidity = [d["humidity"] for d in data]
        wind_speed = [d["wind_speed"] for d in data]
        conditions = [d["condition"] for d in data]

        # 创建图表
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle(f'{weather_data["location"]} 天气预报 - {weather_data["days"]}天', fontsize=16)

        # 温度趋势图
        ax1.plot(dates, temps_min, 'b-o', label='最低温度', linewidth=2)
        ax1.plot(dates, temps_max, 'r-o', label='最高温度', linewidth=2)
        ax1.plot(dates, temps_avg, 'g-o', label='平均温度', linewidth=2)
        ax1.set_title('温度趋势')
        ax1.set_ylabel('温度 (°C)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.tick_params(axis='x', rotation=45)

        # 湿度图
        ax2.bar(dates, humidity, color='skyblue', alpha=0.7)
        ax2.set_title('湿度')
        ax2.set_ylabel('湿度 (%)')
        ax2.tick_params(axis='x', rotation=45)
        ax2.grid(True, alpha=0.3)

        # 风速图
        ax3.plot(dates, wind_speed, 'orange', marker='s', linewidth=2)
        ax3.set_title('风速')
        ax3.set_ylabel('风速 (km/h)')
        ax3.tick_params(axis='x', rotation=45)
        ax3.grid(True, alpha=0.3)

        # 天气状况分布
        condition_counts = {}
        for condition in conditions:
            condition_counts[condition] = condition_counts.get(condition, 0) + 1

        ax4.pie(condition_counts.values(), labels=condition_counts.keys(), autopct='%1.1f%%')
        ax4.set_title('天气状况分布')

        plt.tight_layout()

        # 保存图表
        filename = f"weather_chart_{weather_data['location']}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.show()

        return f"天气图表已生成：{filename}"

    except Exception as e:
        return f"生成图表失败：{str(e)}"

print("✓ 天气可视化工具创建完成")

## 3. 创建智能体

In [ ]:
# 从环境变量读取配置
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")
model_name = os.getenv("MODEL_NAME", "gpt-3.5-turbo")

# 初始化 LLM
llm = ChatOpenAI(
    model=model_name,
    temperature=0,
    api_key=SecretStr(api_key),
    base_url=base_url,
)

print("✓ LLM 初始化完成")

In [ ]:
# 创建智能体提示词模板
prompt = ChatPromptTemplate.from_messages([
    ("system", """
    你是一个专业的天气助手智能体。你能够：

    1. 获取指定城市的天气信息
    2. 分析天气数据并提供建议
    3. 生成天气可视化图表
    4. 根据天气情况给出穿衣、出行建议

    可用工具：
    - get_weather: 获取天气数据
    - visualize_weather: 生成天气图表

    工作流程：
    1. 理解用户需求
    2. 获取相关天气数据
    3. 分析数据并生成可视化
    4. 提供个性化建议

    请用中文回答，保持友好和专业的语调。
    """),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

print("✓ 智能体提示词模板创建完成")

In [ ]:
# 创建工具列表
tools = [get_weather, visualize_weather]

# 创建智能体
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

print("✓ 天气智能体创建完成")

In [ ]:
# 创建智能体执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # 显示详细执行过程
    max_iterations=5  # 最大迭代次数
)

print("✓ 智能体执行器创建完成")

## 4. 测试天气智能体

In [ ]:
# 测试 1: 基本天气查询
print("=== 测试 1: 基本天气查询 ===")
response1 = agent_executor.invoke({
    "input": "查询北京明天的天气情况"
})
print(f"\n最终回答：{response1['output']}")

In [ ]:
# 测试 2: 多天天气预报和可视化
print("\n=== 测试 2: 多天天气预报和可视化 ===")
response2 = agent_executor.invoke({
    "input": "请查询上海未来3天的天气，并生成可视化图表"
})
print(f"\n最终回答：{response2['output']}")

# 显示生成的图表文件（如果存在）
if "图表已生成" in response2['output']:
    print("\n📊 天气图表已生成并保存！")

In [ ]:
# 测试 3: 穿衣建议
print("\n=== 测试 3: 穿衣建议 ===")
response3 = agent_executor.invoke({
    "input": "明天我需要带伞吗？我在广州"
})
print(f"\n最终回答：{response3['output']}")

In [ ]:
# 测试 4: 复杂查询
print("\n=== 测试 4: 复杂查询 ===")
response4 = agent_executor.invoke({
    "input": "帮我比较一下北京和深圳未来一周的天气趋势，给我一些出行建议"
})
print(f"\n最终回答：{response4['output']}")

## 5. 智能体工作原理分析

In [ ]:
# 分析智能体的决策过程
print("=== 智能体工作原理分析 ===")
print("\n1. 思考过程（Thought）：")
print("   - 智能体分析用户意图")
print("   - 决定需要使用哪些工具")
print("   - 规划执行步骤")

print("\n2. 行动选择（Action）：")
print("   - 选择合适的工具")
print("   - 构造工具参数")
print("   - 执行工具调用")

print("\n3. 观察结果（Observation）：")
print("   - 获取工具返回结果")
print("   - 分析结果质量")
print("   - 决定是否需要进一步操作")

print("\n4. 最终回答（Final Answer）：")
print("   - 综合所有信息")
print("   - 生成用户友好的回答")
print("   - 提供个性化建议")

## 6. 扩展功能

In [ ]:
# 创建天气预警工具
@tool
def check_weather_alert(location: str) -> str:
    """检查指定地点的天气预警信息。

    Args:
        location (str): 城市名称

    Returns:
        str: 预警信息
    """
    try:
        weather_data = get_weather_data(location, 1)
        today = weather_data["data"][0]

        alerts = []

        # 温度预警
        if today["temperature"]["max"] > 35:
            alerts.append("高温预警：最高温度超过35°C")
        elif today["temperature"]["min"] < 0:
            alerts.append("低温预警：最低温度低于0°C")

        # 雨天预警
        if today["rain"]:
            alerts.append("降雨预警：今天有雨，请携带雨具")

        # 风速预警
        if today["wind_speed"] > 20:
            alerts.append("大风预警：风速超过20km/h")

        # 湿度预警
        if today["humidity"] > 85:
            alerts.append("高湿度预警：湿度超过85%，体感闷热")

        if not alerts:
            alerts.append("天气状况良好，无特殊预警")

        return json.dumps({
            "location": location,
            "date": today["date"],
            "alerts": alerts
        }, ensure_ascii=False, indent=2)

    except Exception as e:
        return json.dumps({"error": f"检查预警失败：{str(e)}"}, ensure_ascii=False)

print("✓ 天气预警工具创建完成")

In [ ]:
# 测试预警功能
print("\n=== 测试天气预警功能 ===")
alert_response = check_weather_alert.invoke({"location": "北京"})
alert_data = json.loads(alert_response)

print(f"地点：{alert_data['location']}")
print(f"日期：{alert_data['date']}")
print("预警信息：")
for alert in alert_data['alerts']:
    print(f"  - {alert}")

## 7. 性能监控和优化

In [ ]:
# 创建性能监控函数
import time
from typing import Dict, Any

def monitor_agent_performance(agent_executor, query: str) -> Dict[str, Any]:
    """监控智能体性能"""
    start_time = time.time()

    try:
        result = agent_executor.invoke({"input": query})
        end_time = time.time()

        return {
            "success": True,
            "query": query,
            "response": result['output'],
            "execution_time": round(end_time - start_time, 2),
            "intermediate_steps": len(result.get('intermediate_steps', []))
        }
    except Exception as e:
        end_time = time.time()
        return {
            "success": False,
            "query": query,
            "error": str(e),
            "execution_time": round(end_time - start_time, 2)
        }

# 测试性能监控
test_queries = [
    "北京天气怎么样？",
    "上海未来3天天气并生成图表",
    "我需要带伞吗？"
]

print("\n=== 智能体性能监控 ===")
for query in test_queries:
    performance = monitor_agent_performance(agent_executor, query)
    print(f"\n查询：{query}")
    print(f"成功：{performance['success']}")
    print(f"执行时间：{performance['execution_time']}秒")
    if performance['success']:
        print(f"中间步骤：{performance['intermediate_steps']}步")
    else:
        print(f"错误：{performance['error']}")

## 总结

在这个示例中，我们学习了：

1. **Agent 基础**: 智能体的概念和工作原理
2. **工具创建**: 使用 @tool 装饰器创建自定义工具
3. **工具调用**: 利用 LangChain 1.x 的工具调用机制
4. **数据获取**: 模拟天气 API 数据获取
5. **数据可视化**: 使用 matplotlib 生成天气图表
6. **智能体执行**: AgentExecutor 管理智能体运行
7. **性能监控**: 监控和分析智能体性能

### 关键概念
- **Agent**: 能够自主使用工具完成复杂任务的 AI 系统
- **Tool**: 智能体可以调用的功能模块
- **AgentExecutor**: 管理智能体执行流程的组件
- **Tool Calling**: 让 LLM 能够调用外部函数的机制
- **Thought-Action-Observation**: 智能体的决策循环

### 智能体优势
- **自主决策**: 根据任务需求选择合适的工具
- **多步推理**: 能够进行复杂的多步骤任务
- **工具集成**: 轻松集成各种外部 API 和服务
- **可扩展性**: 容易添加新的工具和功能

### 实际应用
- **智能客服**: 集成订单查询、退换货等功能
- **个人助理**: 日程管理、邮件发送、信息查询
- **数据分析**: 数据获取、处理、可视化一体化
- **物联网控制**: 设备状态查询、远程控制等

### 下一步
尝试创建自己的智能体，集成更多实用工具！